In [ ]:
import pandas as pd 

data_eas = pd.read_csv('C:/Users/seraf/Downloads/EAS MACHINE LEARNING/EAS LSTM/dataset.csv')

In [ ]:
data_eas

In [ ]:
import folium
from folium.plugins import FastMarkerCluster
import numpy as np
from IPython.display import display
# Generate random coordinates (replace with your data)

lats = data_eas['Latitude']
lons = data_eas['Longitude']

# Combine latitude and longitude into a list of lists
#heat_data = list(zip(lats, lons))

# Add a heatmap layer
#HeatMap(heat_data, radius=15).add_to(mymap)

# Create a DataFrame from the coordinates
df = pd.DataFrame({'lat': lats, 'lon': lons})

# Downsample the data to reduce the number of points
downsampled_df = df.sample(frac=0.3)  # Adjust the fraction as needed

# Create a Folium map centered around the mean of coordinates
map_center = [np.mean(downsampled_df['lat']), np.mean(downsampled_df['lon'])]
mymap = folium.Map(location=map_center, zoom_start=9)

# Add downsampled data points to the map
for _, row in downsampled_df.iterrows():
    folium.CircleMarker(location=[row['lat'], row['lon']], radius=1, color='blue', fill=True, fill_color='blue').add_to(mymap)

# Display the map
display(mymap)

In [ ]:

#!pip install wandb -qqq
#import wandb
# Log in to your W&B account
#wandb.login()
import sys
import numpy as np 
import math
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D, Flatten
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, Callback, EarlyStopping
from cond_rnn import ConditionalRecurrent
from wandb.keras import WandbCallback

# Import data
data_path = 'C:/Users/seraf/Downloads/EAS MACHINE LEARNING/data/data_seq/'
x_train = np.load(data_path + '/x_train.npz')
x_test = np.load(data_path + '/x_test.npy')
x_valid = np.load(data_path + '/x_valid.npy')
y_train = np.load(data_path + '/y_train.npy')
y_test = np.load(data_path + '/y_test.npy')
y_valid = np.load(data_path + '/y_valid.npy')

x_train = np.asarray(x_train['arr_0']).astype(np.float32)
# Create inputs sets

#0:'Max_Draught'
#1: 'Latitude'
#2: 'Longitude'
#3: 'Speed_over_Ground'
#4: 'COG_cos'
#5: 'COG_sin'
#6: 'TH_cos'
#7: 'TH_sin'
#8: 'Navigational_Status_0.0'
#9: 'Navigational_Status_1.0'
#10: 'Navigational_Status_2.0'
#11: 'Navigational_Status_3.0'
#12: 'Navigational_Status_4.0'
#13: 'Na13vigational_Status_5.0'
#14: 'Nav14igational_Status_8.0'
#15: 'Navi15gational_Status_15.0'
#16: 'GT'
#17: 'DWT'
#18: 'LOA'
#19: 'BEAM'
#20: 'VesselTypeB_Cargo'
#21: 'VesselTypeB_Tanker'
#22: 'Age'
#23: 'current_utotal' 
#24: 'current_vtotal'
#25: 'wind_u10'
#26: 'wind_v10'
#27: 'mwd' 
#28: 'mwp' 
#29: 'swh'
#30: 'sst'  
#31: 'Origin_Lat'
#32: 'Origin_Lon'
#33: 'acc_dist' 
#34: 'acc_time_hours'
#35: 'leg_distance'
#36: 'leg_speed'
#37: 'leg_elapsed_time_hours'
#38: 'remaining_distance'

# Data split of sequence and non sequence variables

### Inputs set: All varaibles -- AIS + Vessel particulars + Weather + Crafted features ###
###########################################################################################
#sequence variables
x_train_nn_cc= np.delete(x_train,[16,17,18,19,20,21,22,31,32],2)
x_test_nn_cc= np.delete(x_test,[16,17,18,19,20,21,22,31,32],2)
x_valid_nn_cc= np.delete(x_valid,[16,17,18,19,20,21,22,31,32],2)

#non sequence variables 
x_train_cc_split= np.delete(x_train,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,23,24,25,26,27,28,29,30,33,34,35,36,37,38],2)
x_test_cc_split= np.delete(x_test,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,23,24,25,26,27,28,29,30,33,34,35,36,37,38],2)
x_valid_cc_split= np.delete(x_valid,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,23,24,25,26,27,28,29,30,33,34,35,36,37,38],2)

### Inputs set: AIS + Vessel particulars ###
###################################################
#sequence variables

#x_train_nn_cc= np.delete(x_train,[16,17,18,19,20,21,22,31,32,23,24,25,26,27,28,29,30,33,34,35,36,38],2)
#x_test_nn_cc= np.delete(x_test,[16,17,18,19,20,21,22,31,32,23,24,25,26,27,28,29,30,33,34,35,36,38],2)
#x_valid_nn_cc= np.delete(x_valid,[16,17,18,19,20,21,22,31,32,23,24,25,26,27,28,29,30,33,34,35,36,38],2)

#non sequence variables 

#x_train_cc_split= np.delete(x_train,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,23,24,25,26,27,28,29,30,33,34,35,36,37,38,31,32],2)
#x_test_cc_split= np.delete(x_test,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,23,24,25,26,27,28,29,30,33,34,35,36,37,38,31,32],2)
#x_valid_cc_split= np.delete(x_valid,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,23,24,25,26,27,28,29,30,33,34,35,36,37,38,31,32],2)

### Inputs set: AIS + Vessel particulars ###
###################################################
#sequence variables

#x_train_nn_cc= np.delete(x_train,[16,17,18,19,20,21,22,31,32,33,34,35,36,38],2)
#x_test_nn_cc= np.delete(x_test,[16,17,18,19,20,21,22,31,32,33,34,35,36,38],2)
#x_valid_nn_cc= np.delete(x_valid,[16,17,18,19,20,21,22,31,32,33,34,35,36,38],2)

#non sequence variables 
#x_train_cc_split= np.delete(x_train,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,23,24,25,26,27,28,29,30,33,34,35,36,37,38,31,32],2)
#x_test_cc_split= np.delete(x_test,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,23,24,25,26,27,28,29,30,33,34,35,36,37,38,31,32],2)
#x_valid_cc_split= np.delete(x_valid,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,23,24,25,26,27,28,29,30,33,34,35,36,37,38,31,32],2)

### Inputs set: AIS + Vessel particulars ###
###################################################
#sequence variables

#x_train_nn_cc= np.delete(x_train,[16,17,18,19,20,21,22,31,32,23,24,25,26,27,28,29,30],2)
#x_test_nn_cc= np.delete(x_test,[16,17,18,19,20,21,22,31,32,23,24,25,26,27,28,29,30],2)
#x_valid_nn_cc= np.delete(x_valid,[16,17,18,19,20,21,22,31,32,23,24,25,26,27,28,29,30],2)

#non sequence variables 

#x_train_cc_split= np.delete(x_train,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,23,24,25,26,27,28,29,30,33,34,35,36,37,38],2)
#x_test_cc_split= np.delete(x_test,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,23,24,25,26,27,28,29,30,33,34,35,36,37,38],2)
#x_valid_cc_split= np.delete(x_valid,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,23,24,25,26,27,28,29,30,33,34,35,36,37,38],2)


print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_valid shape:', x_valid.shape)
print('y_valid shape:', y_valid.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)
print('')
print('x_train_nn_cc shape:', x_train_nn_cc.shape)
print('x_valid_nn_cc shape:', x_valid_nn_cc.shape)
print('x_test_nn_cc shape:', x_test_nn_cc.shape)
print('')
print('x_train_cc shape:', x_train_cc_split.shape)
print('x_valid_cc shape:', x_valid_cc_split.shape)
print('x_test_cc shape:', x_test_cc_split.shape)

n_features_nn_cc = x_train_nn_cc.shape[2]
n_features_cc = x_train_cc_split.shape[2]
steps = x_train.shape[1]
print('n_features_nn_cc = ', n_features_nn_cc)
print('n_features_cc = ', n_features_cc)
print('steps = ', steps)


test_seq_len = np.load(data_path + '/test_sequence_length.npy')


In [ ]:
#get non sequence data into the right shape
def get_cc_data(data):
  X = []
  for i in range(data.shape[0]):
    x = data[i,0,:]
    x = x.reshape (1,9) 
    X.append(x)
  X = np.array(X).reshape(data.shape[0],data.shape[2])
  return X  

x_train_cc = get_cc_data(x_train_cc_split)
x_valid_cc = get_cc_data(x_valid_cc_split)
x_test_cc = get_cc_data(x_test_cc_split)


In [ ]:
test_preds = []
def train(n):
    # Default values for hyper-parameters we're going to sweep over

    config_try = {
        'epochs': n,
        'batch_size': 64,
        'log_learning_rate': -2.4773670403758667,
        'lstm_size': 50,
        'dropout_out': 0.2,
        'dropout_inp': 0.2,
        'dropout_rec':0.1,
        'seed': 40
    }

    # Config is a variable that holds and saves hyperparameters and inputs
    config = config_try
    
    # Define the model architecture 
    checkpoint_filepath = f"folds.hdf5"
    i = Input(shape=(steps, n_features_nn_cc))
    ic = Input(shape=(n_features_cc,))
    m = ConditionalRecurrent(LSTM(config['lstm_size'], return_sequences=True, dropout=config['dropout_out'], recurrent_dropout=config['dropout_rec']))([i, ic])
    m = Dropout(config['dropout_rec'])(m)
    m = TimeDistributed(Dense(1, activation = 'relu'))(m) 
    model = Model([i, ic], m)  
    lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
    es = EarlyStopping(monitor="val_loss", patience=20, verbose=1, mode="min", restore_best_weights=True)
    sv = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
        save_weights_only=False, mode='auto', save_freq='epoch',
        options=None
    )

    model.compile(optimizer=keras.optimizers.Adam(learning_rate = 10**config['log_learning_rate']), loss='mse', metrics=['mae'])

         # Train the model and get the training history
    history = model.fit([x_train_nn_cc, x_train_cc], y_train, batch_size=config['batch_size'],
                        epochs=config['epochs'],
                        shuffle=False,
                        validation_data=([x_valid_nn_cc, x_valid_cc], y_valid),
                        callbacks=[lr,es])
    train_loss, train_mae = model.evaluate([x_train_nn_cc, x_train_cc], y_train, verbose=2)
    valid_loss, valid_mae = model.evaluate([x_valid_nn_cc, x_valid_cc], y_valid, verbose=2)
    test_loss, test_mae = model.evaluate([x_test_nn_cc, x_test_cc], y_test, verbose=2)
    
    model.save("C:/Users/seraf/OneDrive/Documents/lstm_EAS.h5")
    return model, history


# Call the train function
trained_model, training_history = train(300)



In [ ]:
import matplotlib.pyplot as plt
# Plot the training loss over epochs

plt.plot(training_history.history['loss'])
plt.title('Training Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

# Plot the validation loss over epochs
plt.plot(training_history.history['val_loss'])
plt.title('Validation Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('val_loss')
plt.show()


In [ ]:
#plot y_true vs y_predictions from test set
def plot_y_true_y_pred(name, model, x_test, y_test):
  test_predictions = model.predict(x_test)

  plt.figure(figsize=[2, 2], dpi=150)
  a = plt.axes(aspect='equal')

  plt.scatter(y_test, test_predictions, s=0.1)
  plt.xticks(fontsize=5)
  plt.yticks(fontsize=5)
  plt.xlabel('True Values [hour]', fontsize=7)
  plt.ylabel('Predictions [hour]', fontsize=7)
  plt.title(name, fontsize=8)
  lims = [0, 10]
  plt.xlim(lims)
  plt.ylim(lims)
  _ = plt.plot(lims, lims)

## LSTM Dengan Cuaca

In [ ]:
#load the models
models_path = data_path = 'C:/Users/seraf/OneDrive/Documents/'
lstm_EAS = keras.models.load_model(models_path + 'lstm_EAS.h5', custom_objects={'ConditionalRecurrent': ConditionalRecurrent})


In [ ]:
plot_y_true_y_pred('LSTM', lstm_EAS, [x_test_nn_cc, x_test_cc], y_test)

In [ ]:
#calculate the mse per obseravtion of sequence models for comparison with non sequence models 
def mse_obs(x_data_nn_cc, x_data_cc, y_data, model, seq_len):
  """
    Calculates the mse per observation of the sequence models
    
    Arguments:
    x_data_nn_cc -- sequential data
    x_data_cc -- non sequential data
    y_data -- true target
    model -- predictive model
    seq_len -- the true sequence length of the example

    Returns:
    squared error -- list of mse per observation
    MSE -- mean squared error
    """
    
  squared_error = []

  for i in range(x_data_nn_cc.shape[0]):
    y = y_data[i].reshape(1,steps,1)
    z = seq_len[i]
    y_hat = model.predict([x_data_nn_cc[i].reshape(1,steps,n_features_nn_cc), x_data_cc[i].reshape(1,n_features_cc)])
    error = np.square(y[:,:z,:] - y_hat[:,:z,:])
    error = error.reshape(z)
    error = error.tolist()
    squared_error.extend(error)
 
  return np.mean(squared_error), squared_error

In [ ]:

#calculate the MAE per obseravtion of sequence models for comparison with non sequence models 
def mae_obs(x_data_nn_cc, x_data_cc, y_data, model, seq_len) :
  """
    Calculates the mse per observation of the sequence models
    
    Arguments:
    x_data_nn_cc -- sequential data
    x_data_cc -- non sequential data
    y_data -- true target
    model -- predictive model
    seq_len -- the true sequence length of the example

    Returns:
    absolute error -- list of mse per observation
    MAE -- mean absolute error
    """
  absolute_error =[]
  for i in range(x_data_nn_cc.shape[0]):
    y = y_data[i].reshape(1,steps,1)
    z = seq_len[i]
    y_hat = model.predict([x_data_nn_cc[i].reshape(1,steps,n_features_nn_cc),x_data_cc[i].reshape(1,n_features_cc)])
    error = np.abs(y[:,:z,:] - y_hat[:,:z,:])
    error = error.reshape(z)
    error = error.tolist()
    absolute_error.extend(error)
  return np.mean(absolute_error), absolute_error

In [ ]:
#calculate mse for sequence models
lstm_cuaca_mse_obs = mse_obs(x_test_nn_cc, x_test_cc, y_test, lstm_EAS, test_seq_len)



#calculate mae for sequence models
lstm_cuaca_mae_obs = mae_obs(x_test_nn_cc, x_test_cc, y_test, lstm_EAS, test_seq_len)


In [ ]:

print('The performance metrics on the test set are:')
print('')
print('LSTM_1:')
print('The MSE considering errors per observation:', lstm_cuaca_mse_obs[0])

print('')

print('The performance metrics on the test set are:')
print('')
print('LSTM_1:')
print('MAE considering errors per observation:', lstm_cuaca_mae_obs[0] )


## Tanpa Cuaca

In [ ]:

#!pip install wandb -qqq
#import wandb
# Log in to your W&B account
#wandb.login()
import sys
import numpy as np 
import math
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D, Flatten
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, Callback, EarlyStopping
from cond_rnn import ConditionalRecurrent
from wandb.keras import WandbCallback

# Import data
data_path = 'C:/Users/seraf/Downloads/EAS MACHINE LEARNING/data/data_seq/'
x_train = np.load(data_path + '/x_train.npz')
x_test = np.load(data_path + '/x_test.npy')
x_valid = np.load(data_path + '/x_valid.npy')
y_train = np.load(data_path + '/y_train.npy')
y_test = np.load(data_path + '/y_test.npy')
y_valid = np.load(data_path + '/y_valid.npy')

x_train = np.asarray(x_train['arr_0']).astype(np.float32)


#sequence variables
x_train_nn_cc= np.delete(x_train,[16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32],2)
x_test_nn_cc= np.delete(x_test,[16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32],2)
x_valid_nn_cc= np.delete(x_valid,[16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32],2)

#non sequence variables 
x_train_cc_split= np.delete(x_train,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,23,24,25,26,27,28,29,30,33,34,35,36,37,38],2)
x_test_cc_split= np.delete(x_test,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,23,24,25,26,27,28,29,30,33,34,35,36,37,38],2)
x_valid_cc_split= np.delete(x_valid,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,23,24,25,26,27,28,29,30,33,34,35,36,37,38],2)



print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_valid shape:', x_valid.shape)
print('y_valid shape:', y_valid.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)
print('')
print('x_train_nn_cc shape:', x_train_nn_cc.shape)
print('x_valid_nn_cc shape:', x_valid_nn_cc.shape)
print('x_test_nn_cc shape:', x_test_nn_cc.shape)
print('')
print('x_train_cc shape:', x_train_cc_split.shape)
print('x_valid_cc shape:', x_valid_cc_split.shape)
print('x_test_cc shape:', x_test_cc_split.shape)

n_features_nn_cc = x_train_nn_cc.shape[2]
n_features_cc = x_train_cc_split.shape[2]
steps = x_train.shape[1]
print('n_features_nn_cc = ', n_features_nn_cc)
print('n_features_cc = ', n_features_cc)
print('steps = ', steps)


test_seq_len = np.load(data_path + '/test_sequence_length.npy')


In [ ]:
#get non sequence data into the right shape
def get_cc_data(data):
  X = []
  for i in range(data.shape[0]):
    x = data[i,0,:]
    x = x.reshape (1,9) 
    X.append(x)
  X = np.array(X).reshape(data.shape[0],data.shape[2])
  return X  

x_train_cc = get_cc_data(x_train_cc_split)
x_valid_cc = get_cc_data(x_valid_cc_split)
x_test_cc = get_cc_data(x_test_cc_split)


In [ ]:
test_preds = []
def train(n):
    # Default values for hyper-parameters we're going to sweep over

    config_try = {
        'epochs': n,
        'batch_size': 64,
        'log_learning_rate': -2.4773670403758667,
        'lstm_size': 50,
        'dropout_out': 0.2,
        'dropout_inp': 0.2,
        'dropout_rec':0.1,
        'seed': 40
    }

    # Config is a variable that holds and saves hyperparameters and inputs
    config = config_try
    
    # Define the model architecture 
    checkpoint_filepath = f"folds.hdf5"
    i = Input(shape=(steps, n_features_nn_cc))
    ic = Input(shape=(n_features_cc,))
    m = ConditionalRecurrent(LSTM(config['lstm_size'], return_sequences=True, dropout=config['dropout_out'], recurrent_dropout=config['dropout_rec']))([i, ic])
    m = Dropout(config['dropout_rec'])(m)
    m = TimeDistributed(Dense(1, activation = 'relu'))(m) 
    model = Model([i, ic], m)  
    lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
    es = EarlyStopping(monitor="val_loss", patience=20, verbose=1, mode="min", restore_best_weights=True)
    sv = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
        save_weights_only=False, mode='auto', save_freq='epoch',
        options=None
    )

    model.compile(optimizer=keras.optimizers.Adam(learning_rate = 10**config['log_learning_rate']), loss='mse', metrics=['mae'])

         # Train the model and get the training history
    history = model.fit([x_train_nn_cc, x_train_cc], y_train, batch_size=config['batch_size'],
                        epochs=config['epochs'],
                        shuffle=False,
                        validation_data=([x_valid_nn_cc, x_valid_cc], y_valid),
                        callbacks=[lr,es])
    train_loss, train_mae = model.evaluate([x_train_nn_cc, x_train_cc], y_train, verbose=2)
    valid_loss, valid_mae = model.evaluate([x_valid_nn_cc, x_valid_cc], y_valid, verbose=2)
    test_loss, test_mae = model.evaluate([x_test_nn_cc, x_test_cc], y_test, verbose=2)
    
    model.save("C:/Users/seraf/OneDrive/Documents/lstm_EAS_tnp_cuaca.h5")
    return model, history


# Call the train function
trained_model, training_history = train(300)



In [ ]:
import matplotlib.pyplot as plt
# Plot the training loss over epochs

plt.plot(training_history.history['loss'])
plt.title('Training Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

# Plot the validation loss over epochs
plt.plot(training_history.history['val_loss'])
plt.title('Validation Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('val_loss')
plt.show()


In [ ]:
#load the models
models_path = data_path = 'C:/Users/seraf/OneDrive/Documents/'
lstm_EAS_tnp_cuaca = keras.models.load_model(models_path + 'lstm_EAS_tnp_cuaca.h5', custom_objects={'ConditionalRecurrent': ConditionalRecurrent})


In [ ]:
plot_y_true_y_pred('LSTM Tanpa Cuaca', lstm_EAS_tnp_cuaca, [x_test_nn_cc, x_test_cc], y_test)

In [ ]:
#calculate the mse per obseravtion of sequence models for comparison with non sequence models 
def mse_obs(x_data_nn_cc, x_data_cc, y_data, model, seq_len):
  """
    Calculates the mse per observation of the sequence models
    
    Arguments:
    x_data_nn_cc -- sequential data
    x_data_cc -- non sequential data
    y_data -- true target
    model -- predictive model
    seq_len -- the true sequence length of the example

    Returns:
    squared error -- list of mse per observation
    MSE -- mean squared error
    """
    
  squared_error = []

  for i in range(x_data_nn_cc.shape[0]):
    y = y_data[i].reshape(1,steps,1)
    z = seq_len[i]
    y_hat = model.predict([x_data_nn_cc[i].reshape(1,steps,n_features_nn_cc), x_data_cc[i].reshape(1,n_features_cc)])
    error = np.square(y[:,:z,:] - y_hat[:,:z,:])
    error = error.reshape(z)
    error = error.tolist()
    squared_error.extend(error)
 
  return np.mean(squared_error), squared_error

In [ ]:

#calculate the MAE per obseravtion of sequence models for comparison with non sequence models 
def mae_obs(x_data_nn_cc, x_data_cc, y_data, model, seq_len) :
  """
    Calculates the mse per observation of the sequence models
    
    Arguments:
    x_data_nn_cc -- sequential data
    x_data_cc -- non sequential data
    y_data -- true target
    model -- predictive model
    seq_len -- the true sequence length of the example

    Returns:
    absolute error -- list of mse per observation
    MAE -- mean absolute error
    """
  absolute_error =[]
  for i in range(x_data_nn_cc.shape[0]):
    y = y_data[i].reshape(1,steps,1)
    z = seq_len[i]
    y_hat = model.predict([x_data_nn_cc[i].reshape(1,steps,n_features_nn_cc),x_data_cc[i].reshape(1,n_features_cc)])
    error = np.abs(y[:,:z,:] - y_hat[:,:z,:])
    error = error.reshape(z)
    error = error.tolist()
    absolute_error.extend(error)
  return np.mean(absolute_error), absolute_error

In [ ]:
#calculate mse for sequence models
lstm_tdk_cuaca_mse_obs = mse_obs(x_test_nn_cc, x_test_cc, y_test, lstm_EAS_tnp_cuaca, test_seq_len)



#calculate mae for sequence models
lstm_tdk_cuaca_mae_obs = mae_obs(x_test_nn_cc, x_test_cc, y_test, lstm_EAS_tnp_cuaca, test_seq_len)


In [ ]:

print('The performance metrics on the test set are:')
print('')
print('LSTM_1:')
print('The MSE considering errors per observation:', lstm_tdk_cuaca_mse_obs[0])

print('')

print('The performance metrics on the test set are:')
print('')
print('LSTM_1:')
print('MAE considering errors per observation:', lstm_tdk_cuaca_mae_obs[0] )
